# Exercise 4: Multivariate analysis

For this exercise we recommend using [SciKit-Learn](https://scikit-learn.org/stable/index.html)

## Machine Learning for the $\tau^-\to\mu^-\mu^+\mu^-$ search at LHCb

The LHCb experiment at CERN is able to search for exotic decays of the $\tau$ lepton to three muons. This process is fobidden in the Standard Model but is predicted to be sizeable in several theories beyond the Standard Model.

Your goal is to maximise the chances to find this decay in the dataset collected by LHCb using a Machine Learning technique to classify the recorded events as signal or background.

You are given a dataset composed of labelled signal (`signal==1`) and background (`signal==0`) events. Each event is described by a set of *features* whose distribution might be different between signal and background. The goal is to train a Machine Learning classifier to guess if an event is signal or background based on these features.

Here's the list of features recorded in the dataset that you can use for the classification:
* `DecayTime` - How long the $\tau$ candidate existed before decaying.
* `IP` - [Impact parameter](https://en.wikipedia.org/wiki/Impact_parameter) of the $\tau$ candidate and the collision point.
* `VertexChi2` - The $\chi^2$ of a fit to locate the $\tau$ decay vertex.
* `pt` - Transverse momentum of the $\tau$.
* `DOCAone` - Distance of closest approach between first and second muons.
* `DOCAtwo` - Distance of closest approach between second and third muons.
* `DOCAthree` - Distance of closest approach between first and third muons.
* `isolationa` - Track isolation variable.
* `isolationb` - Track isolation variable.
* `isolationc` - Track isolation variable.
* `p0_pt` - Transverse momentum of the first muon.
* `p1_pt` - Transverse momentum of the second muon.
* `p2_pt` - Transverse momentum of the third muon.
* `p0_IP` - Impact parameter of the first muon.
* `p1_IP` - Impact parameter of the second muon.
* `p2_IP` - Impact parameter of the third muon.

In [2]:
import pandas as pd
d = pd.read_csv('training_reduced.csv')


1. Calculate the [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) between signal and background for each of the features of the dataset. The test evaluates how likely are two data samples to be drawn from the same PDF, and therefore should be larger for the features that are most powerful to discriminate signal and background.

*Hint:* you can use the [`ks_2samp` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html) from `scipy.stats` to compute the Kolmogorov-Smirnov test between two samples

*Hint:* you can get the list of features from the DataFrame column headings, i.e. `d.keys()`

In [10]:
from scipy.stats import ks_2samp
#help(ks_2samp)
keys = d.keys()
for key in keys:
    # print(d[key][d["signal"] == 0])
    print(ks_2samp(d[key][d["signal"] == 0], d[key][d["signal"] == 1]))

KstestResult(statistic=1.0, pvalue=0.0)
KstestResult(statistic=0.14423987132737376, pvalue=2.5633875826106553e-290)
KstestResult(statistic=0.5503323429451006, pvalue=0.0)
KstestResult(statistic=0.28567312506181025, pvalue=0.0)
KstestResult(statistic=0.1377976891036844, pvalue=6.7965246029508e-265)
KstestResult(statistic=0.15488706886849868, pvalue=0.0)
KstestResult(statistic=0.15537702908242712, pvalue=0.0)
KstestResult(statistic=0.14469064866972225, pvalue=3.8510728182137147e-292)
KstestResult(statistic=0.24067798381021005, pvalue=0.0)
KstestResult(statistic=0.16782063583131873, pvalue=0.0)
KstestResult(statistic=0.1686414213935396, pvalue=0.0)
KstestResult(statistic=0.10270530235899855, pvalue=4.5061441238451737e-147)
KstestResult(statistic=0.1370815349196095, pvalue=3.850658338917715e-262)
KstestResult(statistic=0.039081124813342416, pvalue=1.2683241258042925e-21)
KstestResult(statistic=0.1283789911393617, pvalue=7.5456630574203215e-230)
KstestResult(statistic=0.01844486715444149, p

2. The least and most discriminating features are the ones with the lowest and highest Kolmogorov-Smirnov scores, respectively. Compare the signal and background distributions for these two features.

**NB:** please do not use the label `signal` as a feature!

3. Split the dataset *randomly* in two halves: we will use one half for training our classifier and the other half to test it.

*Hint:* you can use the function `train_test_split` from `sklearn.model_selection`

**NB:** the dataset is ordered by `signal`, hence the need for random splitting.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(d.drop("signal", axis=1), d["signal"], test_size = 0.5)


4.

i. Train a [Gaussian naive Bayes classifier](https://scikit-learn.org/stable/modules/naive_bayes.html) on the training sample using the three features of your choice (possibly the most discriminant ones). You can use [`GaussianNB` from `sklearn.naive_bayes`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

ii. Now train a [gradient-boosted decision tree (GBDT)](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting) on the training sample using the same three features. You can use [`GradientBoostingClassifier` from `sklearn.ensemble`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

5. Calculate the classifier responses on both the training and testing samples and overlay histograms of the response for signal and background.

*Hint:* to evaluate the response use `gbdt.decision_function()` and `gnb.predict_proba()`

*Hint:* the `predict_proba()` function will return a 2D array: background & signal probabilities for each point in the sample. You can call `predict_proba(...)[:,0]` or `predict_proba(...)[:,1]` to obtain just the signal score.

6. Check if there is overtraining by performing a Kolmogorov-Smirnov test on the classifier responses between the training and testing sample. If you did not overtrain the classifier, its response should be the same on the training and testing samples both for signal and for background. Therefore the K-S scores should be small (~0.01).

7. Show the performance of your classifiers with a ROC curve. Plot the ROC curves on the same plot and label the axes.

*Hint:* you can use the function `roc_curve` from `sklearn.metrics`

In [ ]:
from sklearn.metrics import roc_curve
#help(roc_curve)

**Bonus**: Try to improve the performance of your classifier by including more features and changing the initialisation options of the `GradientBoostingClassifier` object.